<div style="border:solid gray 2px; padding: 30px">
    
Составление сводной таблицы рейтинга и количества пользователей различных "приложений-помощников на основе ИИ". Данные взяты с сайта https://chromewebstore.google.com/ с помощью технологии парсинга.

#### **1. Загрузка библиотек**

In [1]:
import pandas as pd
import requests
import lxml
from bs4 import BeautifulSoup as bs
import re
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

#### 2. Парсинг кода со страницы сайта

In [2]:
url = ('https://chromewebstore.google.com/detail/liner-chatgpt-ai-copilot/bmhcbmnbenmcecpmpepghooflbehcack')

In [3]:
page = requests.get(url)
soup = bs(page.text)

#### 3. Парсинг отдельных элементов со страницы сайта

#### 3.1 Наименование сайта

* вывод наименования приложения через select по первому элементу

In [4]:
soup.select('h1[class="Pa2dE"]')[0].text

'Liner: ChatGPT AI Copilot for Web&YouTube&PDF'

* вывод наименования приложения через find по первому элементу который встретит по ключу

In [5]:
def get_title(soup_page):
    return soup_page.find('h1', {'class':"Pa2dE"}).text

In [6]:
get_title(soup)

'Liner: ChatGPT AI Copilot for Web&YouTube&PDF'

#### 3.2 Средняя оценка приложения от пользователей

* поиск значения по ключу и перевод в тип float

In [7]:
def get_rating(soup_page):
    rating = soup_page.find('span', {'class':"Vq0ZA"}).text
    return float(rating)

In [8]:
get_rating(soup)

4.4

#### 3.3 Количество поставленных оценок

* поиск количества проставленных оценок и перевод в формат числа

In [9]:
def get_user_rate(soup_page):
    user_rate = soup_page.find('p', {'class':"xJEoWe"}).text
    if 'K ratings' in user_rate:
        result = user_rate.replace('K ratings','')
        result = float(result)*1000
        return int(result)

In [10]:
get_user_rate(soup)

6000

#### 3.4 Количество пользователей приложения

* поиск количества пользователей и перевод в формат числа

In [11]:
def get_user_numbers(soup_page):
    users = soup_page.find('div', {'class':'F9iKBc'}).text
    numbers = re.findall(r'\d+', users)
    users_number = ''.join(str(number) for number in numbers)
    return int(users_number)

In [12]:
get_user_numbers(soup)

500000

#### 4. Функция для нескольких сайтов

In [13]:
def parsedate(url):
           
    page = requests.get(url)
    soup = bs(page.text)  
    
    return {'title':get_title(soup), 
            'rating':get_rating(soup), 
            'user_rate':get_user_rate(soup), 
            'users_number':get_user_numbers(soup)}

parsedate(url)

{'title': 'Liner: ChatGPT AI Copilot for Web&YouTube&PDF',
 'rating': 4.4,
 'user_rate': 6000,
 'users_number': 500000}

In [14]:
url_2 = ('https://chromewebstore.google.com/detail/chatgpt-writer-write-mail/pdnenlnelpdomajfejgapbdpmjkfpjkp')
parsedate(url_2)

{'title': 'ChatGPT Writer - Write mail, messages with AI',
 'rating': 4.6,
 'user_rate': 1300,
 'users_number': 700000}

In [15]:
url_3 = ('https://chromewebstore.google.com/detail/monica-ai-copilot-%D0%BD%D0%B0-%D0%B1%D0%B0%D0%B7%D0%B5/ofpnmcalabcbjgholdjcjblkibolbppb')
parsedate(url_3)

{'title': 'Monica - Your AI Copilot powered by ChatGPT4',
 'rating': 4.9,
 'user_rate': 16300,
 'users_number': 2000000}

#### 5. Выбор приложений и текущее время анализа данных, вывод сводной таблицы

In [16]:
now = datetime.now()

competitors = ['https://chromewebstore.google.com/detail/liner-chatgpt-ai-copilot/bmhcbmnbenmcecpmpepghooflbehcack', 
               'https://chromewebstore.google.com/detail/chatgpt-writer-write-mail/pdnenlnelpdomajfejgapbdpmjkfpjkp', 
               'https://chromewebstore.google.com/detail/monica-ai-copilot-%D0%BD%D0%B0-%D0%B1%D0%B0%D0%B7%D0%B5/ofpnmcalabcbjgholdjcjblkibolbppb',
               'https://chromewebstore.google.com/detail/sider-chatgpt-%D0%B1%D0%BE%D0%BA%D0%BE%D0%B2%D0%B0%D1%8F-%D0%BF%D0%B0%D0%BD/difoiogjjojoaoomphldepapgpbgkhkb',
               'https://chromewebstore.google.com/detail/merlin-ask-ai-to-research/camppjleccjaphfdbohjdohecfnoikec']

lst = []

for competitor in competitors:
    res = parsedate(competitor)
    res['update_at'] = now
    lst.append(res)
    
df = pd.DataFrame(lst)
df

,title,rating,user_rate,users_number,update_at
0,Liner: ChatGPT AI Copilot for Web&YouTube&PDF,4.4,6000,500000,2024-06-24 12:32:54.217588
1,"ChatGPT Writer - Write mail, messages with AI",4.6,1300,700000,2024-06-24 12:32:54.217588
2,Monica - Your AI Copilot powered by ChatGPT4,4.9,16300,2000000,2024-06-24 12:32:54.217588
3,"Sider: ChatGPT Sidebar + GPT-4o, Claude 3.5, G...",4.9,29900,2000000,2024-06-24 12:32:54.217588
4,"Merlin - Ask AI to Research, Write & Review",4.8,8600,1000000,2024-06-24 12:32:54.217588
